In [2]:
import pdaggerq
import sys
sys.path.append('#PATH_TO_SimpleWick')
import wick as w
import copy
import interface_pdaggerq as ipq
from IPython.display import display, Latex

# Don't forget to kill/reload after changes in interface_pdaggerq or SimpleWick...

In [3]:
# T limited to single and double excitations
max_rank_t = 2
# Indices used for the active orbitals
list_act_idx = ['m','n']
# alpha/beta spin
spin = ['a','b']

# TD CC
## T1 equations
$$Q_i^a(A) = (<A_i^a| e^{^AT}|A>)_c - (<A_i^a|e^{^BT}|B> H_{BA}^{\text{eff}})_c $$

In the following block, one can see a list:
```
A_ia = ['in+aa','im+ba','am-aa','an-ba','@(oi)i+@(si)i','@(oa)a-@(sa)i']
```
It's a string of second quantized operators defined for SimpleWick. 
The format is explained in the documentation of SimpleWick.
In addition, the 'ipq.monoK_eTL_L' function will replace '@(oi)' by the 
two possible occupancies, similarly for '@(oa)'. A similar treatment is also 
done with '@(si)' and '(@sa)' for the spin.

So, 'A_ia' is just $\hat{n}_{i_\alpha}^\dagger \hat{m}_{i_\beta}^\dagger \hat{m}_{a_\alpha}^{} \hat{n}_{a_\beta}^{} \hat{i}_{}^\dagger \hat{a}_{}^{}$, where $\hat{i}$ and $\hat{a}$ will take all the 
possible combinations of occupancy and spin.

Then, the 'ipq.monoK_eTL_L' function is applying Wick's theorem and calling $p^\dagger q$.
The results are stored in 'A_ia_eTB_B' (literally $<A_i^a | e^{^B\hat{T}} | B>$) depending on the spin of $i$ and $a$.
After that, we run over the different spins of $i$ and $a$ to:
1. remove the terms that are not connected with the effective Hamiltonian, i.e., those with an amplitude that does not contain any active label
2. spin flip the amplitudes related to $B$ to keep $A$ as reference
3. factorize the terms
4. sort the terms with respect to $t_1$
5. generate fortran code for each spin case

Fortran code:

You can generate fortran code by removing the # in front of obj.gen_fortran...

As you can see there are lot of indices in the fortran code. 
1. 'ia', 'ib', 'aa', 'ab': indices to loop over the spin orbitals $i_\alpha,i_\beta,a_\alpha,a_\beta$
2. 'ma', 'mb', 'na', 'nb': indices of the active spin orbitals $m_\alpha,m_\beta,n_\alpha,n_\beta$
3. 'cc_nOa', 'cc_nVa': number of occupied and virtual alpha spin orbitals. Since in our software the orbitals
are ordered as: alpha occupied, beta occupied, alpha virtual, beta virtual, we can compute the indice of $i_\beta$ based on $i_\alpha$, as 'ib = ia + cc_nOa'
4. 'i_ia', 'f_ia': indice of the first and last occupied alpha spin orbital
5. 'i_ib', 'f_ib': same for occupied beta spin orbital
6. 'i_aa', 'f_aa': same for virtual alpha spin orbital
7. 'i_ab', 'f_ab': same for virtual beta spin orbital

In order to generate the expressions "i,j,a,b" have been hard coded at some points in the code for the functions "ordered_by_t1" and "reverse_order_deltas". That's not a big deal but these functions will not work with other labels. For the fortran code generation, a significant number of functions use them so it will not work with other labels. Nevertheless the code can be easily changed for that.



### Calculation of $-<A_i^a|e^{^BT}|B>$

In [4]:
A_ia = ['in+aa','im+ba','am-aa','an-ba','@(oi)i+@(si)i','@(oa)a-@(sa)i']
A_ia_eTB_B = ipq.monoK_eTL_L(-1.0,A_ia,'B',max_rank_t)

for i in range(len(spin)):
    si = spin[i]
    for a in range(len(spin)):
        sa = spin[a]
        obj = A_ia_eTB_B[i][a]
        obj.remove_disconnected()
        obj.spin_flip('B','A')
        obj.reverse_deltas_order()
        obj.factorize()
        obj.ordered_by_t1()
        #print("\\begin{equation}\n\\begin{split}")
        null = ipq.print_contrib_M1(si,sa,len(obj.terms))
        obj.show_tex_factorized()
        #print("\\end{split}\n\\end{equation}")
        #obj.gen_fortran_M1(si,sa,'A')

## T2 equations
$Q_{ij}^{ab} = (<A_{ij}^{ab}| e^{^AT}|A>)_c - (<A_{ij}^{ab}| e^{^BT}|B> H_{BA}^{\text{eff}})_c 
- P(ij,ab) \ [ (^At_i^a - R_{ia} \ ^Bt_i^a) \ (<A_j^b|e^{^BT}|B> H_{BA}^{\text{eff}})_c ] \\
= (<A_{ij}^{ab}| e^{^AT}A>)_c + M_{ij}^{ab} H_{BA}^{\text{eff}} $


### Calculation of $-<A_{ij}^{ab}|e^{^BT}|B>$

In [18]:
A_ijab = ['in+aa','im+ba','am-aa','an-ba','@(oi)i+@(si)i','@(oj)j+@(sj)i','@(ob)b-@(sb)i','@(oa)a-@(sa)i']
A_ijab_eTB_B = ipq.doubleK_eTL_L(-1.0,A_ijab,'B',max_rank_t)
spin = ['a','b']
for i in range(len(spin)):
    si = spin[i]
    for j in range(len(spin)):
        sj = spin[j]
        for a in range(len(spin)):
            sa = spin[a]
            for b in range(len(spin)):
                sb = spin[b]
                obj = A_ijab_eTB_B[i][j][a][b]
                obj.remove_disconnected()
                obj.spin_flip('B','A')
                obj.reverse_deltas_order()
                obj.factorize()
                obj.ordered_by_t1()
                #null = ipq.print_contrib_M2(si,sj,sa,sb,len(obj.terms))
                #obj.show_tex_factorized()
                

### Calculation of $<A_i^a | e^{^AT} | A >$

For that we compute $^At_i^A$ for the cases where zero, one or two of the indices are active ($n$ or $m$, with or without bar).

- ipq.T creates a t with some label and spin.
- ipq.Term creates a ensemble of a kronecker delta, a factor and a t.
- obj.append_Term append the Term to the object obj

In [5]:
A_ia_eTA_A = [[ipq.LTerms() for i in spin] for a in spin]

for i in range(len(spin)):
    si = spin[i]
    for a in range(len(spin)):
        sa = spin[a]
        obj = A_ia_eTA_A[i][a]
        if si == sa:
            t = ipq.T(['i'+si,'a'+sa],'A',list_act_idx)
            obj.append_Term(ipq.Term([],1.0,[t]))
        
        obj.factorize()
        #obj.show_tex_factorized()

# t_ia^na
obj = A_ia_eTA_A[0][0]
t = ipq.T(['ia','na'],'A',list_act_idx)
obj.append_Term(ipq.Term([['na','aa']],1.0,[t]))
obj.factorize()
#obj.show_tex_factorized()

# t_ib^ab
obj = A_ia_eTA_A[1][1]
t = ipq.T(['ib','mb'],'A',list_act_idx)
obj.append_Term(ipq.Term([['mb','ab']],1.0,[t]))
obj.factorize()
#obj.show_tex_factorized()

# t_ma^aa
obj = A_ia_eTA_A[0][0]
t = ipq.T(['ma','aa'],'A',list_act_idx)
obj.append_Term(ipq.Term([['ma','ia']],1.0,[t]))
obj.factorize()
#obj.show_tex_factorized()

# t_nb^ab
obj = A_ia_eTA_A[1][1]
t = ipq.T(['nb','ab'],'A',list_act_idx)
obj.append_Term(ipq.Term([['nb','ib']],1.0,[t]))
obj.factorize()
#obj.show_tex_factorized()

# t_ma^na
obj = A_ia_eTA_A[0][0]
t = ipq.T(['ma','na'],'A',list_act_idx)
obj.append_Term(ipq.Term([['ma','ia'],['na','aa']],1.0,[t]))
obj.factorize()
#obj.show_tex_factorized()

# t_nb^mb
obj = A_ia_eTA_A[1][1]
t = ipq.T(['nb','mb'],'A',list_act_idx)
obj.append_Term(ipq.Term([['nb','ib'],['mb','ab']],1.0,[t]))
obj.factorize()
#obj.show_tex_factorized()

# t_alpha^alpha 
print("t_alpha^alpha:")
obj = A_ia_eTA_A[0][0]
obj.factorize()
obj.show_tex_factorized()

# t_beta^beta 
print("t_beta^beta:")
obj = A_ia_eTA_A[1][1]
obj.factorize()
obj.show_tex_factorized()

Here, we apply the permutation operators $P_{ij}$, $P_{ab}$, $P_{ij}$ $P_{ab}$ on the latter quantities.

- apply_permutation apply a permutation ont the t and the kronecker deltas with a factor.

In [6]:
# Permutations
A_ja_eTA_A = [[ipq.LTerms() for i in spin] for a in spin]
A_ib_eTA_A = [[ipq.LTerms() for i in spin] for a in spin]
A_jb_eTA_A = [[ipq.LTerms() for i in spin] for a in spin]

for i in range(len(spin)):
    for a in range(len(spin)):
        A_ja_eTA_A[i][a] = copy.deepcopy(A_ia_eTA_A[i][a])
        A_ja_eTA_A[i][a].apply_permutation(1.0,[['i','j']])
        A_ib_eTA_A[i][a] = copy.deepcopy(A_ia_eTA_A[i][a])
        A_ib_eTA_A[i][a].apply_permutation(1.0,[['a','b']])
        A_jb_eTA_A[i][a] = copy.deepcopy(A_ia_eTA_A[i][a])
        A_jb_eTA_A[i][a].apply_permutation(1.0,[['i','j'],['a','b']])
        

We do a similar treatment for $^Bt_i^a$, but the orbitals cannot be active due to the $R_{ia}$ term

In [7]:
B_ia_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]
B_ja_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]
B_ib_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]
B_jb_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]

for i in range(len(spin)):
    si = spin[i]
    for a in range(len(spin)):
        sa = spin[a]
        obj = B_ia_eTB_B[i][a]
        if si == sa:
            t = ipq.T(['i'+si,'a'+sa],'B',list_act_idx)
            obj.append_Term(ipq.Term([],1.0,[t])) 
        obj.spin_flip('B','A')
        obj.factorize()
        obj.show_tex_factorized()


Here we apply the permutation operators as for $^At_i^a$

In [8]:
# Permutations
B_ja_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]
B_ib_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]
B_jb_eTB_B = [[ipq.LTerms() for i in spin] for a in spin]

for i in range(len(spin)):
    for a in range(len(spin)):
        #print(i,a)
        #print('ja')
        B_ja_eTB_B[i][a] = copy.deepcopy(B_ia_eTB_B[i][a])
        B_ja_eTB_B[i][a].apply_permutation(1.0,[['i','j']])
        obj = B_ja_eTB_B[i][a]
        obj.factorize()
        obj.ordered_by_t1()
        #obj.show_tex_factorized()
        #print('ib')
        B_ib_eTB_B[i][a] = copy.deepcopy(B_ia_eTB_B[i][a])
        B_ib_eTB_B[i][a].apply_permutation(1.0,[['a','b']])
        obj = B_ib_eTB_B[i][a]
        obj.factorize()
        obj.ordered_by_t1()
        #obj.show_tex_factorized()
        #print('jb')
        B_jb_eTB_B[i][a] = copy.deepcopy(B_ia_eTB_B[i][a])
        B_jb_eTB_B[i][a].apply_permutation(1.0,[['i','j'],['a','b']])
        obj = B_jb_eTB_B[i][a]
        obj.factorize()
        obj.ordered_by_t1()
        obj.show_tex_factorized()
        

### Calculation of $<A_j^b|e^{T(B)}|B>$

In [9]:
A_jb = ['in+aa','im+ba','am-aa','an-ba','@(oi)j+@(si)i','@(oa)b-@(sa)i']

A_jb_eTB_B = ipq.monoK_eTL_L(1.0,A_jb,'B',max_rank_t)

for i in range(len(spin)):
    si = spin[i]
    for a in range(len(spin)):
        sa = spin[a]
        obj = A_jb_eTB_B[i][a]
        obj.remove_disconnected()
        obj.spin_flip('B','A')
        obj.factorize()
        obj.ordered_by_t1()
        #obj.show_tex_factorized()

### Calculation of its permutations of $<A_j^b|e^{T(B)}|B>$:
$<A_j^a|e^{T(B)}|B>$, $<A_i^b|e^{T(B)}|B>$, $<A_i^a|e^{T(B)}|B>$

In [10]:
# Permutations
A_ja_eTB_B = [[ipq.LTerms() for j in spin] for b in spin]
A_ib_eTB_B = [[ipq.LTerms() for j in spin] for b in spin]
A_ia_eTB_B = [[ipq.LTerms() for j in spin] for b in spin]

for j in range(len(spin)):
    for b in range(len(spin)):
        A_ja_eTB_B[j][b] = copy.deepcopy(A_jb_eTB_B[j][b])
        A_ja_eTB_B[j][b].apply_permutation(1.0,[['a','b']])
        A_ib_eTB_B[j][b] = copy.deepcopy(A_jb_eTB_B[j][b])
        A_ib_eTB_B[j][b].apply_permutation(1.0,[['i','j']])
        A_ia_eTB_B[j][b] = copy.deepcopy(A_jb_eTB_B[j][b])
        A_ia_eTB_B[j][b].apply_permutation(1.0,[['i','j'],['a','b']])
        
        obj = A_jb_eTB_B[j][b]
        obj.remove_disconnected()
        obj.spin_flip('B','A')
        obj.factorize()
        obj.ordered_by_t1()
        obj.show_tex_factorized()
        

### Calculation of $- P(ij,ab) \ ^At_i^a <A_j^b|e^{^BT}|B>$.

- obj.prod_LTerms produces the products of two lists of Terms with a prefactor and add them to the list of Terms
already present in the object obj.

You can notice that we are in fact computing $+ P(ij,ab) \ ^At_i^a <A_j^b|e^{^BT}|B>$ and similarly computing $- P(ij,ab) \ ^At_i^a <A_j^b|e^{^BT}|B>$. It comes from the fact that with a global sign difference on the disconnected, and we need to do that to recover the right formula.

In [12]:
A = [[[[ipq.LTerms() for i in spin] for j in spin] for a in spin] for b in spin]
for i in range(len(spin)):
    si = spin[i]
    for j in range(len(spin)):
        sj = spin[j]
        for a in range(len(spin)):
            sa = spin[a]
            for b in range(len(spin)):
                sb = spin[b]
                
                A[i][j][a][b] = ipq.LTerms()
                obj = A[i][j][a][b]
                obj.prod_LTerms(+1,A_ia_eTA_A[i][a],A_jb_eTB_B[j][b])
                obj.prod_LTerms(-1,A_ja_eTA_A[j][a],A_ib_eTB_B[i][b])
                obj.prod_LTerms(-1,A_ib_eTA_A[i][b],A_ja_eTB_B[j][a])
                obj.prod_LTerms(+1,A_jb_eTA_A[j][b],A_ia_eTB_B[i][a])
                
                obj.spin_flip('B','A')
                obj.reverse_deltas_order()
                obj.factorize()
                obj.ordered_by_t1()
                #null = ipq.print_contrib_M2(si,sj,sa,sb,len(obj.terms))
                #obj.show_tex_factorized()
                

### Calculation of   $+ P(ij,ab) \ R_{ia} \ ^Bt_i^a <A_j^b|e^{^BT}|B>$

In [13]:
B = [[[[ipq.LTerms() for i in spin] for j in spin] for a in spin] for b in spin]
for i in range(len(spin)):
    si = spin[i]
    for j in range(len(spin)):
        sj = spin[j]
        for a in range(len(spin)):
            sa = spin[a]
            for b in range(len(spin)):
                sb = spin[b]
                
                B[i][j][a][b] = ipq.LTerms()
                obj = B[i][j][a][b]
                obj.prod_LTerms(-1,B_ia_eTB_B[i][a],A_jb_eTB_B[j][b])
                obj.prod_LTerms(+1,B_ja_eTB_B[j][a],A_ib_eTB_B[i][b])
                obj.prod_LTerms(+1,B_ib_eTB_B[i][b],A_ja_eTB_B[j][a])
                obj.prod_LTerms(-1,B_jb_eTB_B[j][b],A_ia_eTB_B[i][a])
                
                obj.spin_flip('B','A')
                obj.reverse_deltas_order()
                obj.factorize()
                obj.ordered_by_t1()
                #null = ipq.print_contrib_M2(si,sj,sa,sb,len(obj.terms))
                #obj.show_tex_factorized()



# Calculation of $^AM_{ij}^{ab}$


## Disconnected contributions
We call disconnected contributions the terms that are coming from 
$P(ij,ab) \ [ (^At_i^a - R_{ia} \ ^Bt_i^a) \ (<A_j^b|e^{^BT}|B> H_{BA}^{\text{eff}})_c$

In [16]:
M2 = [[[[ipq.LTerms() for i in spin] for j in spin] for a in spin] for b in spin]
for i in range(len(spin)):
    si = spin[i]
    for j in range(len(spin)):
        sj = spin[j]
        for a in range(len(spin)):
            sa = spin[a]
            for b in range(len(spin)):
                sb = spin[b]
                
                M2[i][j][a][b] = ipq.LTerms()
                obj = M2[i][j][a][b]
                obj.append_LTerms(A[i][j][a][b])
                obj.append_LTerms(B[i][j][a][b])
                
                #obj.spin_flip('B','A')
                obj.factorize()
                obj.ordered_by_t1()
                #print("\\begin{equation}\n\\begin{split}")
                null = ipq.print_contrib_M2(si,sj,sa,sb,len(obj.terms))
                obj.show_tex_factorized()
                #print("\\end{split}\n\\end{equation}")
                #obj.gen_fortran_M2_disconnected(si,sj,sa,sb,'A')

## Connected contributions
We call connected contribution the terms that are coming from $(<A_{ij}^{ab}| e^{^BT}|B> H_{BA}^{\text{eff}})_c$

In [19]:
M2 = [[[[ipq.LTerms() for i in spin] for j in spin] for a in spin] for b in spin]
for i in range(len(spin)):
    si = spin[i]
    for j in range(len(spin)):
        sj = spin[j]
        for a in range(len(spin)):
            sa = spin[a]
            for b in range(len(spin)):
                sb = spin[b]
                
                M2[i][j][a][b] = ipq.LTerms()
                obj = M2[i][j][a][b]
                obj.append_LTerms(A_ijab_eTB_B[i][j][a][b])
                
                obj.spin_flip('B','A')
                obj.factorize()
                obj.ordered_by_t1()
                #print("\\begin{equation}\n\\begin{split}")
                null = ipq.print_contrib_M2(si,sj,sa,sb,len(obj.terms))
                obj.show_tex_factorized()
                #print("\\end{split}\n\\end{equation}")
                #obj.gen_fortran_M2(si,sj,sa,sb,'A')
